In [4]:
%pip install numpy
%pip install pandas

import numpy as np 
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from rake_nltk import Rake
import warnings
import random
warnings.simplefilter(action = 'ignore', category = FutureWarning)


Note: you may need to restart the kernel to use updated packages.


In [5]:
movie_df = pd.read_csv('./web_movie.csv')
auth_user = pd.read_csv('./auth_user.csv')
ratings_df = pd.read_csv('./web_myrating.csv')
movie_df[['title','director','stars','genre','overview']]

,title,director,stars,genre,overview
0,The_Shawshank_Redemption,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Drama,Two imprisoned men bond over a number of years...
1,Fight_Club,David Fincher,"Brad Pitt, Edward Norton, Meat Loaf, Zach Grenier",Drama,An insomniac office worker and a devil-may-car...
2,"Il_buono,_il_brutto,_il_cattivo",Sergio Leone,"Clint Eastwood, Eli Wallach, Lee Van Cleef, Al...",Western,A bounty hunting scam joins two men in an unea...
3,One_Flew_Over_the_Cuckoo's_Nest,Milos Forman,"Jack Nicholson, Louise Fletcher, Michael Berry...",Drama,A criminal pleads insanity and is admitted to ...
4,Soorarai_Pottru,Sudha Kongara,"Suriya, Madhavan, Paresh Rawal, Aparna Balamurali",Drama,"Nedumaaran Rajangam ""Maara"" sets out to make t..."
...,...,...,...,...,...
87,"Synecdoche,_New_York",Charlie Kaufman,"Philip Seymour Hoffman, Samantha Morton, Miche...",Drama,"A theatre director struggles with his work, an..."
88,Mysterious_Skin,Gregg Araki,"Brady Corbet, Joseph Gordon-Levitt, Elisabeth ...",Drama,A teenage hustler and a young man obsessed wit...
89,25th_Hour,Spike Lee,"Edward Norton, Barry Pepper, Philip Seymour Ho...",Drama,"Cornered by the DEA, convicted New York drug d..."
90,Y_tu_mamá_también,Alfonso Cuarón,"Maribel Verdú, Gael García Bernal, Daniel Gimé...",Drama,"In Mexico, two teenage boys and an attractive ..."


## CONTENT BASED FILTERTING 

In [6]:
movie = movie_df
ratings = ratings_df

In [7]:
movie['stars'] = movie['stars'].map(lambda x: x.split(' ')[:])
movie['genre'] = movie['genre'].map(lambda x: x.lower().split(' '))
movie['director'] = movie['director'].map(lambda x: x.split(' '))

In [8]:
for index, row in movie.iterrows():
    row['stars'] = [x.lower().replace(' ','') for x in row['stars']]
    row['director'] = ''.join(row['director']).lower()
movie['Key_words'] = " "
list_key = []
for index, row in movie.iterrows():
    plot = row['overview']
    r = Rake()
    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    # print(list(key_words_dict_scores.keys()))
    list_key.append(list(key_words_dict_scores.keys()))

movie['Key_words'] = list_key
# movie['Key_words']

In [9]:
movie.drop(columns = ['overview'], inplace = True)
movie.set_index('title', inplace = True)
movie.drop(['movie_id','movie_logo_link','movie_logo','released_year','certificate','runtime','imdb_ratings','score','no_of_votes'],1,inplace=True)
movie['director'] = movie_df['director'].values
# movie

In [10]:
movie['bag_of_words'] = ''
columns = movie.columns
str = []
for index, row in movie.iterrows():
    words = ''
    for col in columns:
        if col != 'director':
            words = words + ' '.join(row[col]) + ' '
        else:
            words = words + ''.join(row[col]) + ' '
    words += "|"
    str.append(words[:-2])
movie['bag_of_words'] = str
# movie['bag_of_words'].values

In [11]:
count = CountVectorizer()
count_matrix = count.fit_transform(movie['bag_of_words'])
indices = pd.Series(movie.index)
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [12]:
def recommendations(title, cosine_sim = cosine_sim):
		recommended_movies = pd.DataFrame()
		# gettin the index of the movie that matches the title
		idx = indices[indices == title].index[0]
		# creating a Series with the similarity scores in descending order
		score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
		# getting the indexes of the 10 most similar movies
		top_10_indexes = list(score_series.index)
		# populating the list with the titles of the best 10 matching movies
		for i in top_10_indexes:
			recommended_movies = recommended_movies.append([[movie.index[i],score_series.loc[i]]])
		return recommended_movies[1:]

In [13]:
movie_df = pd.read_csv("./web_movie.csv")
ratings_df = pd.read_csv("./web_myrating.csv")

In [14]:
user_movie_df = movie_df.merge(ratings_df,how="right",on="movie_id")
random_user_title = user_movie_df.loc[user_movie_df['user_id'] == random.randint(1,4)][['title','user_id','genre']]
random_user_title = random_user_title.values[0]
random_user_title


array(["One_Flew_Over_the_Cuckoo's_Nest", 2, 'Drama'], dtype=object)

In [15]:
content = recommendations(random_user_title[0])
content.columns = ['title','cosine_similarity']
content

,title,cosine_similarity
0,The_Innocents,0.097823
0,Udaan,0.093048
0,Network,0.090909
0,Seven_Pounds,0.090909
0,The_Verdict,0.087039
...,...,...
0,Once_Upon_a_Time_in_the_West,0.000000
0,Wait_Until_Dark,0.000000
0,"Il_buono,_il_brutto,_il_cattivo",0.000000
0,The_Exorcist,0.000000


## COLLABORATIVE BASED FILTERING

In [16]:
raters = ratings_df.groupby(['movie_id'])['rating'].count() ## Number of users rated this movie_id
# raters

In [17]:
data=ratings_df.merge(raters,on='movie_id')
# data

In [18]:
data['count'] = data['rating_y']
data['rating']= data['rating_x']
data.drop(['rating_x','rating_y'],axis=1,inplace=True)
# data

In [19]:
selfjoined = data.merge(data,on='user_id')
selfjoined = selfjoined[selfjoined['movie_id_x']<selfjoined['movie_id_y']]
# selfjoined

In [20]:
selfjoined['r1r2'] = selfjoined['rating_x']*selfjoined['rating_y']
selfjoined['r1square'] = np.square(selfjoined['rating_x'])
selfjoined['r2square'] = np.square(selfjoined['rating_y'])
# selfjoined

In [21]:
aggdata=selfjoined.groupby(['movie_id_x','movie_id_y'])['rating_x','rating_y','r1r2','r1square','r2square','count_x','count_y'].sum()
n=aggdata.shape[0]
aggdata['corelation'] = (n*aggdata['r1r2'] - aggdata['rating_x']*aggdata['rating_y']) / np.sqrt((n * aggdata['r1square']  - np.square(aggdata['rating_x']))*(n * aggdata['r2square']  - np.square(aggdata['rating_y'])))
aggdata.reset_index(inplace=True)

In [22]:
aggdata1=aggdata.merge(movie_df,left_on='movie_id_x',right_on='movie_id')
aggdata2=aggdata1.merge(movie_df,left_on='movie_id_y',right_on='movie_id')

In [23]:
def recommendation(title):
		data =aggdata2[aggdata2['title_x']==title]
		data = data.sort_values(by='corelation',ascending=False)
		return data
collabaritive=recommendation(random_user_title[0])[['title_y','corelation']]
collabaritive

,title_y,corelation
333,Black,1.000000
323,Ladri_di_biciclette,1.000000
399,Vozvrashchenie,1.000000
240,Philadelphia,1.000000
224,I_Am_Sam,1.000000
407,Festen,1.000000
171,Detachment,1.000000
416,Central_do_Brasil,1.000000
451,Il_conformista,1.000000
103,Forushande,1.000000


In [24]:
collabaritive['title'] = collabaritive['title_y'].str.split(' \(').str.get(0)
collabaritive.drop('title_y',axis=1,inplace=True)

In [25]:
# print(collabaritive,content).

## HYBRID FILTERTING

In [26]:
hybrid=content.merge(collabaritive,left_on='title',right_on='title')
hybrid=hybrid[['title','cosine_similarity','corelation']]
hybrid['wcorelation'] = (hybrid['cosine_similarity'] + hybrid['corelation'])/2
hybrid = hybrid.sort_values(by='wcorelation',ascending=False).head(18)[['title','wcorelation']]
hybrid

,title,wcorelation
5,Festen,0.524456
6,Auf_der_anderen_Seite,0.524456
7,Vivre_sa_vie:_Film_en_douze_tableaux,0.523837
9,Detachment,0.523262
10,Ladri_di_biciclette,0.523262
11,Requiem_for_a_Dream,0.522696
19,Black,0.522228
4,8½,0.522059
24,Forushande,0.521320
25,Vozvrashchenie,0.521320
